In [1]:
from transformers import pipeline

summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [22]:
# Read test text
with open("test.txt", "r") as f:
    text = f.readlines()
    # Delete lines with :
    text = [line for line in text if ':' not in line]
    # Delete empty lines
    text = [line for line in text if line.strip()]
    # Add paragraph break after each line
    text = [line + '\n\n' for line in text]
    # Join lines
    text = " ".join(text)


In [23]:
print(text)

Hello. My name is Adam Ragusea. I am not a chef. I am a guy in his kitchen with a camera.And as such, I do not wield my knife like a trained chef. I use it like a normal person,because I am. And I'm now going to show you the normal person strategies I've used through theyears to convert my foods into multiple smaller foods without cutting myself. I can't rememberthe last time I cut myself in the kitchen. I burn myself all the time, but I never cut myself. Hey,is that the new Adam Ragusea custom chef knife available for purchase at adamragusea.com?


 Maybe. We'll start with an onion. Nearly every savory recipe on earth starts with peel andchop and onion. How exactly does one do that? Step one is get off the papery skin. If you wantto make whole slices of onion, you have to make a very shallow cut across the outer layer and thenpeel. But if you're going to chop the onion, you might as well just cut it in half to begin with.Now, it's way easier to reach under any undesirable outer layers

In [9]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

model = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")
tokenizer = T5Tokenizer.from_pretrained("Michau/t5-base-en-generate-headline")
model = model.to(device)

cpu


In [24]:
from nltk.tokenize.texttiling import TextTilingTokenizer


tt = TextTilingTokenizer()
text = tt.tokenize(text)

In [5]:

article = '''
Very early yesterday morning, the United States President Donald Trump reported he and his wife First Lady Melania Trump tested positive for COVID-19. Officials said the Trumps' 14-year-old son Barron tested negative as did First Family and Senior Advisors Jared Kushner and Ivanka Trump.
Trump took to social media, posting at 12:54 am local time (0454 UTC) on Twitter, "Tonight, [Melania] and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!" Yesterday afternoon Marine One landed on the White House's South Lawn flying Trump to Walter Reed National Military Medical Center (WRNMMC) in Bethesda, Maryland.
Reports said both were showing "mild symptoms". Senior administration officials were tested as people were informed of the positive test. Senior advisor Hope Hicks had tested positive on Thursday.
Presidential physician Sean Conley issued a statement saying Trump has been given zinc, vitamin D, Pepcid and a daily Aspirin. Conley also gave a single dose of the experimental polyclonal antibodies drug from Regeneron Pharmaceuticals.
According to official statements, Trump, now operating from the WRNMMC, is to continue performing his duties as president during a 14-day quarantine. In the event of Trump becoming incapacitated, Vice President Mike Pence could take over the duties of president via the 25th Amendment of the US Constitution. The Pence family all tested negative as of yesterday and there were no changes regarding Pence's campaign events.
'''

text =  "headline: " + text

max_len = 256

encoding = tokenizer.encode_plus(text, return_tensors = "pt")
input_ids = encoding["input_ids"].to(device)
attention_masks = encoding["attention_mask"].to(device)

beam_outputs = model.generate(
    input_ids = input_ids,
    attention_mask = attention_masks,
    max_length = 64,
    num_beams = 3,
    early_stopping = True,
)

result = tokenizer.decode(beam_outputs[0])
print(result)

Token indices sequence length is longer than the specified maximum sequence length for this model (4981 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [25]:
summary_segment = []

for segment in text:
    text_in = "headline: " + segment

    encoding = tokenizer.encode_plus(text_in, return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_masks = encoding["attention_mask"].to(device)

    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=10,
        num_beams=3,
        early_stopping=True,
    )

    result = tokenizer.decode(beam_outputs[0])
    obj = {result: segment}
    print(obj)

{'<pad> Adam Ragusea Custom Chef Knife': "Hello. My name is Adam Ragusea. I am not a chef. I am a guy in his kitchen with a camera.And as such, I do not wield my knife like a trained chef. I use it like a normal person,because I am. And I'm now going to show you the normal person strategies I've used through theyears to convert my foods into multiple smaller foods without cutting myself. I can't rememberthe last time I cut myself in the kitchen. I burn myself all the time, but I never cut myself. Hey,is that the new Adam Ragusea custom chef knife available for purchase at adamragusea.com?"}


Token indices sequence length is longer than the specified maximum sequence length for this model (686 > 512). Running this sequence through the model will result in indexing errors


{'<pad> Peel And Chop And Onion Recipe': "\n\n\n Maybe. We'll start with an onion. Nearly every savory recipe on earth starts with peel andchop and onion. How exactly does one do that? Step one is get off the papery skin. If you wantto make whole slices of onion, you have to make a very shallow cut across the outer layer and thenpeel. But if you're going to chop the onion, you might as well just cut it in half to begin with.Now, it's way easier to reach under any undesirable outer layers and peel them off.You still have this inedible bit of stem at the top to slice off. I swear, if someone comments, oh, that's not a stem, it's a tunic or a scape or whatever. I will think of a snappy reply to you, and then I will never post it. You've been warned. At this point I like to stop and clear away all of the inedible bits before I actually start chopping, with the exception of the root end. You have to leave that because it holds all the layers together as you're cutting through them. First th